# DeepXDE - Bernard Cavity

In this notebook i'll be implementing the bernard cavity case. I'll be following the paper Hashemi+2024 and adding some twists to make it work better as i go.

In [1]:
# stdlib
from copy import deepcopy
from functools import reduce

# 3rth party
import fluidfoam as ff
import deepxde as dde
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


# Setting defaults
# dde.config.set_default_float("float64")
torch.set_default_device('cuda:0')
torch._dynamo.config.suppress_errors = True



Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 5080'

In [3]:
rho = 1
mu = 0.001
alpha = 0.005
D = 0.1
L = D
beta = 1.0
g = 9.81

T0 = 0
T_bottom = 1

S0 = (T0 - T0)/(T_bottom - T0)
S_bottom = (T_bottom - T0)/(T_bottom - T0)


assert T_bottom > T0, "For a Benard cell the bottom temperature should be higher than the rest"

Ra = g*beta*(T_bottom-T0)*D**3*rho/(alpha*mu)
Pr = mu/(rho*alpha)
print(f"{Ra = }, {Pr = }")

Ra = 1962.0000000000005, Pr = 0.2


In [3]:
def transform(X, Y):
    """Ansatz to make the spped exactly zero on the boundaries"""
    noslip_ansatz_factor = X[:, 0:1] * (L - X[:, 0:1]) * X[:, 1:2] * (D - X[:, 1:2])
    temperature_ansatz = X[:, 1:2]/D*(S0 + (D - X[:, 1:2])) + S_bottom * (D - X[:, 1:2])/D

    return torch.hstack(
        (
            noslip_ansatz_factor*Y[:, 0:2],  
            1.0 + (X[:, 0:1]**2 + X[:, 1:2]**2)**0.5*Y[:, 2:3],
            temperature_ansatz +  X[:, 1:2] * (D - X[:, 1:2])*Y[:, 3:]
        )
    )

In [4]:
net = dde.maps.FNN([2,] + [256]*6 + [4,], "silu", "Glorot uniform")

CASE = "../simulation_data/bernard_cell_of_v2/"

x, y, z = ff.readmesh(CASE, structured=True)

u, v, w = ff.readvector(CASE, "1000", "U", structured=True)
p = ff.readscalar(CASE, "1000", "p", structured=True)
T = ff.readscalar(CASE, "1000", "T", structured=True)

N = reduce(lambda l, r: l*r, x.shape)

N

Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/owner
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/faces
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/points
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/neighbour
Reading file ../simulation_data/bernard_cell_of_v2/1000/U
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/owner
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/faces
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/points
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/neighbour
Reading file ../simulation_data/bernard_cell_of_v2/1000/p
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/owner
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/faces
Reading file ../simulation_data/bernard_cell_of_v2//constant/polyMesh/points
Reading file ../simulation_data/berna

40000

In [1]:


X = np.hstack([x.reshape(N, 1), y.reshape(N, 1)], dtype=np.float32)
Y = np.hstack([f.reshape(N, 1) for f in (u, v, p, T)], dtype=np.float32)

X = torch.from_numpy(X).requires_grad_(True).to("cuda")
Y = torch.from_numpy(Y).requires_grad_(True).to("cuda")


batch_size = int(N*0.2)


dataset = TensorDataset(X, Y)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cuda'))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


NameError: name 'np' is not defined

In [21]:
net = dde.maps.FNN([2,] + [256]*6 + [4,], "silu", "Glorot uniform")
net.compile()
net.to("cuda")

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss(reduction='mean')

losses = []

In [22]:
torch.set_float32_matmul_precision('high')

In [23]:
torch._dynamo.config.suppress_errors = True

In [28]:
for epoch in tqdm(range(0, 100_000)):

    optimizer.zero_grad()

    for batch_X, batch_Y in data_loader:
        # Forward pass
        outputs = net(batch_X)
        loss = loss_fn(outputs, batch_Y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    with torch.no_grad():
        net.eval()
        outputs = net.forward(X)
        loss = loss_fn(Y, outputs)
        losses.append(loss.item())
        net.train()
        
    if epoch % 1000 == 0:
        print(f"epoch={len(losses)} | loss = {loss.item()}")

        

    if loss.item() <= min(losses) and loss.item() < 0.1:
        print(f"Best loss at epoch: {len(losses)} | loss={loss.item()}. Saving model")
        net_best = deepcopy(net)

  0%|                                                                               | 1/100000 [00:01<29:18:58,  1.06s/it]

epoch=340 | loss = 0.09743539243936539
Best loss at epoch: 340 | loss=0.09743539243936539. Saving model


  0%|                                                                               | 6/100000 [00:05<27:25:35,  1.01it/s]

Best loss at epoch: 345 | loss=0.09671749919652939. Saving model


  0%|                                                                               | 7/100000 [00:06<26:33:21,  1.05it/s]

...

epoch=1440 | loss = 0.040572166442871094


  1%|▉                                                                           | 1201/100000 [19:22<25:45:25,  1.07it/s]

epoch=1540 | loss = 0.1526051163673401


  1%|▉                                                                           | 1285/100000 [20:43<26:32:38,  1.03it/s]


KeyboardInterrupt: 

In [ ]:
next(net.parameters()).dtype

In [ ]:
torch.all(next(net_best.parameters()) == next(net.parameters()))

In [ ]:
with torch.no_grad():
    Y_test = net.forward(X)
    Y_test_best = net_best.forward(X)

In [ ]:
x.shape[:2]

In [ ]:
U, V = Y_test_best[:, :2].T

U = U.reshape(*x.shape[:2])
V = V.reshape(*x.shape[:2])


plt.quiver(x, y, U.to("cpu"), V.to("cpu"))